# OpenAI Basic Tool Calling AI Agent from Scratch
* Collaborators:
    * Roberto Rodriguez @Cyb3rWard0g

In [ ]:
! pip install openai
! pip install python-dotenv
! pip install pydantic

## The Agent Base Class 🤖

In [1]:
class Agent:
    """
    Integrates LLM client, tools, and memory.
    """
    def __init__(self, llm_client=None, tools=None, memory=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = memory

    def run(self):
        """
        Placeholder for the agent's main execution logic.
        """
        pass

## Building the LLM Client 📡

### The OpenAI Chat Completion Class

In [3]:
from typing import Dict, Any, List
import openai

class OpenAIChatCompletion:
    """
    Interacts with OpenAI's API for chat completions.
    """
    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        """
        Initialize with model, API key, and base URL.
        """
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(self, messages: List[Dict], **kwargs) -> Dict[str, Any]:
        """
        Generate a response from input messages.
        """
        params = {'messages': messages, 'model': self.model, **kwargs}
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

#### 🚦Assembly Line: Integrating LLM Client with AI Agent 🤖

In [4]:
class Agent:
    """
    Integrates LLM client, tools, and memory.
    """
    def __init__(self, llm_client, tools=None, memory=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = memory

    def run(self, messages: List[Dict[str, str]]):
        """
        Generates a response from the LLM client.
        """
        # Generate response using the LLM client
        response = self.llm_client.generate(messages)
        return response

### Testing the Updated Agent Class 🔬

To securely handle the API key, set the `OPENAI_API_KEY` variable in a .env file and use [Python-dotenv](https://pypi.org/project/python-dotenv/) to load it as an environment variable. This method avoids hardcoding sensitive information in your source code. The OpenAI library will automatically detect and use this variable.

In [5]:
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env.

True

Next, initialize the new class, define a few messages, and run the agent:

In [6]:
# Initialize the OpenAI client
client = OpenAIChatCompletion(base_url='https://api.openai.com/v1', model='gpt-4')

# Define a set of messages
messages = [
    {"role": "system", "content": "You are a security assistant."},
    {"role": "user", "content": "Hey! This is Roberto!"}
]

# Initialize the agent with the LLM client
myAgent = Agent(llm_client=client)

# Use the agent to run a task
response = myAgent.run(messages=messages)
print(response)

ChatCompletionMessage(content='Hello Roberto! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)


## Implementing Short-Term Memory 💾

### The Short-Term Memory Class

In [7]:
from typing import List, Dict

class ChatMessageMemory:
    """Manages conversation context."""
    
    def __init__(self):
        self.messages = []
    
    def add_message(self, message: Dict):
        """Add a message to memory."""
        self.messages.append(message)
    
    def add_messages(self, messages: List[Dict]):
        """Add multiple messages to memory."""
        for message in messages:
            self.add_message(message)
    
    def add_conversation(self, user_message: Dict, assistant_message: Dict):
        """Add a user-assistant conversation."""
        self.add_messages([user_message, assistant_message])
    
    def get_messages(self) -> List[Dict]:
        """Retrieve all messages."""
        return self.messages.copy()
    
    def reset_memory(self):
        """Clear all messages."""
        self.messages = []

### 🚦Assembly Line: Integrating Memory with AI Agent 🤖

With the LLM client set up and the memory class ready, the next step was to add memory into the Agent class. Now, the agent initializes with the system message, and user messages are added to memory before merging with the system message. This allows efficient generation, storage, and retrieval of conversation context.

In [8]:
class Agent:
    """Integrates LLM client, tools, and memory."""
    
    def __init__(self, llm_client, system_message: Dict[str, str], tools=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = ChatMessageMemory()
        self.system_message = system_message

    def run(self, user_message: Dict[str, str]):
        """Generate a response using LLM client and store context."""
        self.memory.add_message(user_message)
        chat_history = [self.system_message] + self.memory.get_messages()
        response = self.llm_client.generate(chat_history)
        self.memory.add_message(response)
        return response

### Testing the Updated Agent Class 🔬

In [9]:
# Test the updated agent
client = OpenAIChatCompletion(base_url='https://api.openai.com/v1', model='gpt-4')

# Define the system message
system_message = {"role": "system", "content": "You are a security assistant."}

# Initialize the Agent with the LLM client and system message
agent = Agent(llm_client=client, system_message=system_message)

# Define a user message
user_message = {"role": "user", "content": "Hey! This is Roberto!"}

# Generate a response using the agent
response = agent.run(user_message)

# Add a follow-up message to test memory integration
follow_up_message = {"role": "user", "content": "What was my name?"}

# Run the agent again with the new message
response = agent.run(follow_up_message)
print(response)

ChatCompletionMessage(content='Your name is Roberto.', role='assistant', function_call=None, tool_calls=None)


## Enabling Tool Execution 🛠️

### Testing OpenAI's Function Calling Capability 🔍

1️⃣ Update Open AI Chat Completion Client

First, I needed to update the client to accept an additional tools parameter, allowing users to pass the tool details along with the user message.

In [10]:
from typing import Dict, Any, List
import openai

class OpenAIChatCompletion:
    """Interacts with OpenAI's API for chat completions."""
    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(self, messages: List[str], tools: List[Dict[str, Any]] = None, **kwargs) -> Dict[str, Any]:
        """Generates a response from OpenAI's API."""
        params = {'messages': messages, 'model': self.model, 'tools': tools, **kwargs}
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

2️⃣ Update Agent Class to Integrate Tools

Next, I updated the Agent class to initialize it with tools and pass those tools to the LLM client.

In [11]:
from typing import Dict

class Agent:
    """Integrates LLM client, tools, and memory."""
    def __init__(self, llm_client, system_message: Dict[str, str], tools=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = ChatMessageMemory()
        self.system_message = system_message

    def run(self, user_message: Dict[str, str]):
        self.memory.add_message(user_message)
        chat_history = [self.system_message] + self.memory.get_messages()
        response = self.llm_client.generate(chat_history, tools=self.tools)
        self.memory.add_message(response)
        return response

3️⃣ Define Dummy Python Functions

I defined a few dummy Python functions to test these capabilities.

In [12]:
def get_weather(location: str) -> str:
    """Gets weather information."""
    return f"{location}: 80F."

def jump(distance: str) -> str:
    """Jumps a specific distance."""
    return f"I jumped the following distance {distance}"

4️⃣ Define the Function Signature as OpenAI Function Calls

I then defined a dictionary for each tool following the expected schema.

In [13]:
get_weather_func_dict = {
    'type': 'function',
    'function': {
        'name': 'get_weather',
        'description': 'Get weather information based on location.',
        'parameters': {
            'properties': {'location': {'type': 'string'}},
            'required': ['location'],
            'type': 'object'
        }
    }
}

jump_func_dict = {
    'type': 'function',
    'function': {
        'name': 'jump',
        'description': 'Jump a specific distance.',
        'parameters': {
            'properties': {'distance': {'type': 'string'}},
            'required': ['distance'],
            'type': 'object'
        }
    }
}

5️⃣ Initialize Agent and Test It

I then initialized the client and used it to generate a response from the LLM, including tools.

In [14]:
client = OpenAIChatCompletion(base_url='https://api.openai.com/v1', model='gpt-4')

# Define the system message
system_message = {"role": "system", "content": "You are a security assistant."}

# Gather all available tools
tools = [get_weather_func_dict, jump_func_dict]

# Initialize the Agent with the LLM client, system message, and tools
agent = Agent(llm_client=client, system_message=system_message, tools=tools)

# Define a user message
user_message = {"role": "user", "content": "What is the weather in Virginia?"}

# Generate a response using the agent
response = agent.run(user_message)
response

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_HFyUnaAmRc9trG4HdBwdjg7v', function=Function(arguments='{\n  "location": "Virginia"\n}', name='get_weather'), type='function')])

6️⃣ Capture Assistant's Response and Extract Tool Details

The LLM returned a message with the tool details and suggested arguments.

In [15]:
import json

tool_response = response.tool_calls[0]
tool_arguments = json.loads(tool_response.function.arguments)
tool_arguments


{'location': 'Virginia'}

7️⃣ Execute the Tool with the Function Arguments

I then executed the tool with the suggested arguments.

In [16]:
tool_execution_results = get_weather(**tool_arguments)
tool_execution_results

'Virginia: 80F.'

8️⃣ Craft a Tool Message and Send It to the LLM

Finally, I crafted a Tool message with the results and sent it to the LLM for a final response.

In [17]:
tool_message = {
    "role": "tool",
    "tool_call_id": tool_response.id,
    "name": tool_response.function.name,
    "content": str(tool_execution_results)
}

final_response = agent.run(tool_message)
final_response

ChatCompletionMessage(content='The current weather in Virginia is 80°F.', role='assistant', function_call=None, tool_calls=None)

## Enter Pydantic: Validating Tool Arguments 🔍

[Pydantic](https://docs.pydantic.dev/latest/) is a powerful library that leverages Python type hints for efficient data validation and serialization. Using [Pydantic models](https://docs.pydantic.dev/latest/concepts/models/), we can define and validate tool arguments, ensuring correct data types and structure.

### Define Tool Argument Schema as a Pydantic Model

In [18]:
# Python function
def get_weather(location: str) -> str:
    """Get weather information based on location."""
    return f"{location}: 80F."

# Pydantic Model
from pydantic import BaseModel, Field

class GetWeatherSchema(BaseModel):
    """Get weather information based on location."""
    location: str = Field(description="Location to get weather for")

### Validate Suggested Tool Argument Schema

To validate the suggested tool argument schema, follow the previous steps up to initializing the agent and generating a response.

6️⃣ Capture Assistant's Response and Extract Tool Details

The LLM returned a message with the tool details and suggested arguments.

In [19]:
# Extract tool arguments (JSON string)
tool_response = response.tool_calls[0]
tool_arguments = tool_response.function.arguments

Instead of manually converting the JSON string, validate it directly with Pydantic's [model_validate_json()](https://docs.pydantic.dev/latest/api/base_model/#pydantic.BaseModel.model_validate_json) utility.

In [20]:
response_model = GetWeatherSchema.model_validate_json(tool_arguments)
response_model

GetWeatherSchema(location='Virginia')

You can also generate JSON schemas from Pydantic models using Pydantic's [model_json_schema()](https://docs.pydantic.dev/latest/api/base_model/#pydantic.BaseModel.model_json_schema) utility.

In [21]:
GetWeatherSchema.model_json_schema()

{'description': 'Get weather information based on location.',
 'properties': {'location': {'description': 'Location to get weather for',
   'title': 'Location',
   'type': 'string'}},
 'required': ['location'],
 'title': 'GetWeatherSchema',
 'type': 'object'}

## Formatting Tools for OpenAI Function Calls with Pydantic Models 🔥

### From Pydantic Models to OpenAI Functions 

I used the previous Pydantic model GetWeatherSchema and created a function to convert the tool's argument schema into OpenAI's Function Calling format.

In [22]:
def to_openai_function_call_definition(name: str, model: BaseModel):
    schema_dict = model.model_json_schema()
    description = schema_dict.pop("description", "")
    schema_dict.pop("title", None)  # Remove the title field to exclude the model name
    return {
        "type": "function",
        "function": {
            "name": name,
            "description": description,
            "parameters": schema_dict
        }
    }

Testing this new capability.

In [23]:
function_call_definition = to_openai_function_call_definition("get_weather", GetWeatherSchema)
function_call_definition

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': 'Get weather information based on location.',
  'parameters': {'properties': {'location': {'description': 'Location to get weather for',
     'title': 'Location',
     'type': 'string'}},
   'required': ['location'],
   'type': 'object'}}}

## Streamlining the Agent Tool Concept ⚒️

To streamline the process of converting Python functions to OpenAI's Function Calling format, I created a dedicated class.

### The Agent Tool Class

I developed the AgentTool class to encapsulate a Python function and its Pydantic model. This class includes methods for converting the tool to OpenAI's format and generating a JSON schema of the model.

In [24]:
from pydantic import BaseModel
from typing import Callable, Type

class AgentTool:
    """Encapsulates a Python function with Pydantic validation."""
    def __init__(self, func: Callable, args_model: Type[BaseModel]):
        self.func = func
        self.args_model = args_model
        self.name = func.__name__
        self.description = func.__doc__ or self.args_schema.get('description', '')

    def to_openai_function_call_definition(self) -> dict:
        """Converts the tool to OpenAI Function Calling format."""
        schema_dict = self.args_schema
        description = schema_dict.pop("description", "")
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": description,
                "parameters": schema_dict
            }
        }

    @property
    def args_schema(self) -> dict:
        """Returns the tool's function argument schema as a dictionary."""
        schema = self.args_model.model_json_schema()
        schema.pop("title", None)
        return schema

### The Tool Decorator

In [25]:
from typing import Callable, Optional, Type
from pydantic import BaseModel

def check_docstring(func: Callable):
    """Ensure the function has a docstring."""
    if not func.__doc__:
        raise ValueError(f"Function '{func.__name__}' must have a docstring.")

def Tool(func: Optional[Callable] = None, *, args_model: Type[BaseModel]) -> AgentTool:
    """Decorator to wrap a function with an AgentTool instance."""
    def decorator(f: Callable) -> AgentTool:
        check_docstring(f)
        return AgentTool(f, args_model=args_model)
    return decorator(func) if func else decorator


You could then use the Tool decorator to create an AgentTool and convert it to OpenAI's Function Calling format.

In [26]:
# Python function arguments schema
from pydantic import BaseModel, Field

class GetWeatherSchema(BaseModel):
    location: str = Field(description="location to get weather for")

@Tool(args_model=GetWeatherSchema)
def get_weather(location: str) -> str:
    """Get weather information based on location."""
    return f"{location}: 80F."

# Convert the AgentTool to OpenAI's Function Calling format
get_weather.to_openai_function_call_definition()

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': '',
  'parameters': {'properties': {'location': {'description': 'location to get weather for',
     'title': 'Location',
     'type': 'string'}},
   'required': ['location'],
   'type': 'object'}}}

### Validating Tool Arguments before Agent Tool Execution

To ensure tool arguments were validated before execution, I added functionality to the AgentTool class. This involved creating a method to validate JSON strings using Pydantic Pydantic's model_validate_json() utility and modifying the call function to validate parameters before executing the tool.

In [28]:
from pydantic import BaseModel, ValidationError
from typing import Callable, Type
from inspect import signature

class AgentTool:
    """Encapsulates a Python function with Pydantic validation."""
    def __init__(self, func: Callable, args_model: Type[BaseModel]):
        self.func = func
        self.args_model = args_model
        self.name = func.__name__
        self.description = func.__doc__ or self.args_schema.get('description', '')

    def to_openai_function_call_definition(self) -> dict:
        """Converts the tool to OpenAI Function Calling format."""
        schema_dict = self.args_schema
        description = schema_dict.pop("description", "")
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": description,
                "parameters": schema_dict
            }
        }

    @property
    def args_schema(self) -> dict:
        """Returns the tool's function argument schema as a dictionary."""
        schema = self.args_model.model_json_schema()
        schema.pop("title", None)
        return schema

    def validate_json_args(self, json_string: str) -> bool:
        """Validate JSON string using the Pydantic model."""
        try:
            validated_args = self.args_model.model_validate_json(json_string)
            return isinstance(validated_args, self.args_model)
        except ValidationError:
            return False

    def run(self, *args, **kwargs) -> Any:
        """Execute the function with validated arguments."""
        try:
            # Handle positional arguments by converting them to keyword arguments
            if args:
                sig = signature(self.func)
                arg_names = list(sig.parameters.keys())
                kwargs.update(dict(zip(arg_names, args)))

            # Validate arguments with the provided Pydantic schema
            validated_args = self.args_model(**kwargs)
            return self.func(**validated_args.model_dump())
        except ValidationError as e:
            raise ValueError(f"Argument validation failed for tool '{self.name}': {str(e)}")
        except Exception as e:
            raise ValueError(f"An error occurred during the execution of tool '{self.name}': {str(e)}")

    def __call__(self, *args, **kwargs) -> Any:
        """Allow the AgentTool instance to be called like a regular function."""
        return self.run(*args, **kwargs)

With the new methods in the AgentTool class, you can now validate tool arguments and execute the function with validated parameters. Similar to the previous example, initialize an AgentTool with its schema model using the Tool decorator. Then, validate a dummy tool argument JSON string and run the tool:

In [29]:
# Python function arguments schema
from pydantic import BaseModel, Field

class GetWeatherSchema(BaseModel):
    location: str = Field(description="location to get weather for")

@Tool(args_model=GetWeatherSchema)
def get_weather(location: str) -> str:
    """Get weather information based on location."""
    return f"{location}: 80F."

In [30]:
tool_json_arguments = '{\n  "location": "Virginia"\n}'

# Validate the JSON string arguments
get_weather.validate_json_args(tool_json_arguments)  # True

# Convert JSON string to dictionary and run the tool
tool_arguments_dict = json.loads(tool_json_arguments)
get_weather.run(**tool_arguments_dict)

'Virginia: 80F.'

## Operationalizing Tool Execution ⚒️ -> ⚡️

After setting up our agent to select and prepare tools with the help of LLMs, the next step was to manage these tools effectively. It was essential to have a system that could handle tool availability and execution seamlessly.

### The Agent Tool Executor Class

I created an AgentToolExecutor class to manage the agent's interaction with tools. This class ensures tools are registered, managed, and executed properly, bridging the gap between choosing the right tool and executing it effectively.

In [46]:
from typing import Any, Dict, List, Optional

class AgentToolExecutor:
    """Manages tool registration and execution."""
    
    def __init__(self, tools: Optional[List[AgentTool]] = None):
        self.tools: Dict[str, AgentTool] = {}
        if tools:
            for tool in tools:
                self.register_tool(tool)
    
    def register_tool(self, tool: AgentTool):
        """Registers a tool."""
        if tool.name in self.tools:
            raise ValueError(f"Tool '{tool.name}' is already registered.")
        self.tools[tool.name] = tool
      
    def execute(self, tool_name: str, *args, **kwargs) -> Any:
        """Executes a tool by name with given arguments."""
        tool = self.tools.get(tool_name)
        if not tool:
            raise ValueError(f"Tool '{tool_name}' not found.")
        try:
            return tool(*args, **kwargs)
        except Exception as e:
            raise ValueError(f"Error executing tool '{tool_name}': {e}") from e
    
    def get_tool_names(self) -> List[str]:
        """Returns a list of all registered tool names."""
        return list(self.tools.keys())
    
    def get_tool_details(self) -> str:
        """Returns details of all registered tools."""
        tools_info = [f"{tool.name}: {tool.description} Args schema: {tool.args_schema['properties']}" for tool in self.tools.values()]
        return '\n'.join(tools_info)

### 🚦Assembly Line: Integrating Tool Executor with AI Agent 🤖

In [47]:
import logging
from typing import Dict, List, Optional

logger = logging.getLogger(__name__)

class Agent:
    """Integrates LLM client, tools, memory, and manages tool executions."""
    
    def __init__(self, llm_client, system_message: Dict[str, str], max_iterations: int = 10, tools: Optional[List[AgentTool]] = None):
        self.llm_client = llm_client
        self.executor = AgentToolExecutor()
        self.memory = ChatMessageMemory()
        self.system_message = system_message
        self.max_iterations = max_iterations
        self.tool_history = []
        self.function_calls = None
        
        # Register and convert tools
        if tools:
            for tool in tools:
                self.executor.register_tool(tool)
            self.function_calls = [tool.to_openai_function_call_definition() for tool in tools]

    def run(self, user_message: Dict[str, str]):
        """Generates responses, manages tool calls, and updates memory."""
        self.memory.add_message(user_message)

        for _ in range(self.max_iterations):
            chat_history = [self.system_message] + self.memory.get_messages() + self.tool_history
            response = self.llm_client.generate(chat_history, tools=self.function_calls)

            if self.parse_response(response):
                continue
            else:
                self.memory.add_message(response)
                self.tool_history = []
                return response

    def parse_response(self, response) -> bool:
        """Executes tool calls suggested by the LLM and updates tool history."""
        import json
        
        if response.tool_calls:
            self.tool_history.append(response)
            for tool in response.tool_calls:
                tool_name = tool.function.name
                tool_args = tool.function.arguments
                tool_args_dict = json.loads(tool_args)
                try:
                    logger.info(f"Executing {tool_name} with args: {tool_args}")
                    execution_results = self.executor.execute(tool_name, **tool_args_dict)
                    self.tool_history.append({
                        "role": "tool",
                        "tool_call_id": tool.id,
                        "name": tool_name,
                        "content": str(execution_results)
                    })
                except Exception as e:
                    raise ValueError(f"Execution error in tool '{tool_name}': {e}") from e
            return True
        return False

To test the new Agent base class, follow the steps in previous examples. Initialize the client, define your tools with the Tool decorator, initialize the agent with those tools and the LLM client, and finally, run the agent with your query.

In [48]:
# Set Environment Variables
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

# Initialize LLM Client
client = OpenAIChatCompletion(base_url='https://api.openai.com/v1', model='gpt-4')

# Initialize Agent with tools and LLM client

# Python function arguments schema
from pydantic import BaseModel, Field

class GetWeatherSchema(BaseModel):
    location: str = Field(description="location to get weather for")

@Tool(args_model=GetWeatherSchema)
def get_weather(location: str) -> str:
    """Get weather information based on location."""
    return f"{location}: 80F."

class JumpSchema(BaseModel):
    distance: str = Field(description="distance to jump.")

@Tool(args_model=JumpSchema)
def jump(distance: str) -> str:
    """Jumps a specific distance."""
    return f"I jumped the following distance {distance}"

tools = [jump, get_weather]

# Define the system message
system_message = {"role": "system", "content": "You are a security assistant."}

# Initialize the Agent with the LLM client, system message, and tools
agent = Agent(llm_client=client, system_message=system_message, tools=tools)

# Run Task
# Define a user message
user_message = {"role": "user", "content": "What is the weather in Virginia?"}

# Generate a response using the agent
response = agent.run(user_message)
response

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Executing get_weather with args: {
  "location": "Virginia"
}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletionMessage(content='The weather in Virginia is currently 80°F.', role='assistant', function_call=None, tool_calls=None)

### Would it Handle Multiple Tool Executions? 😎

You can adjust the user message to ask a question that will require a tool to run multiple times. You can enable logging to see tools being executed.

In [49]:
# Enable Logging
import logging
logging.basicConfig(level=logging.INFO)

# Reset previous interactions
agent.memory.reset_memory()

# Define a new task
user_message = {"role": "user", "content": "What is the weather in Virginia, Washington and New York?"}

# Generate a response using the agent
response = agent.run(user_message)
response

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Executing get_weather with args: {
  "location": "Virginia"
}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Executing get_weather with args: {
  "location": "Washington"
}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Executing get_weather with args: {
  "location": "New York"
}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletionMessage(content='The weather in Virginia, Washington, and New York is currently 80F.', role='assistant', function_call=None, tool_calls=None)

In [50]:
agent.memory.get_messages()

[{'role': 'user',
  'content': 'What is the weather in Virginia, Washington and New York?'},
 ChatCompletionMessage(content='The weather in Virginia, Washington, and New York is currently 80F.', role='assistant', function_call=None, tool_calls=None)]